# Advanced plugin development for NMRPro
This tutorial dicusses special cases in plugins development where the [basic tutorial](For_developers.ipynb) falls short. Please be sure to read the [basic tutorial](For_developers.ipynb) first.

Here, we discuss the following 4 points:
1. Customizing input elements
2. Combining multiple functions in one jsCommand using `Include`
3. Specifying function order in the workflow.
4. R functions as plugins

## 1. Customizing input elements
### 1.1 Calling function with default arguments
Let's go back to our simple example from the basic tutorial

In [ ]:
@jsCommand(path=['Processing', 'FFT'], nd=[1,2])  
@interaction(method=('positive', 'normalized'))
@perSpectrum
@perDimension
def fft(s, method='positive'):  
    # Function body 

By default, `jsCommand` looks for `interaction` decorator to create the GUI for function parameters. In some cases we might want to create multiple commands for the same function. For example

In [ ]:
@jsCommand(path=['Processing', 'FFT auto'], nd=[1,2], args=None)  

By specifying `args=None`, no GUI is created for the user. When the user clicks on the command, the function is called with the defaults parameter values.

### 1.2 Specifying changable parameters
Also, we can specify different GUI for each command, using `args` in `jsCommand`. To demonestrate this, let's modify our example to include another parameter

In [ ]:
@jsCommand(path=['Processing', 'FFT'], nd=[1,2])  
@interaction(method=('positive', 'normalized'), swap_halves=False)  
@perSpectrum
@perDimension
def fft(s, method='positive', swap_halves=False):  
    # Function body 

The above `jsCommand` allows users to select the method from a dropdown menu, and whether to swap the halves of the spectrum (displayed as a checkbox). We can create our custom command, that gives the user the ability to specifiy the method only. For this, we will use a special class `ArgsPanel`

In [ ]:
from nmrpro.plugins.JSinput import ArgsPanel
@jsCommand(path=['Processing', 'FFT'], nd=[1,2],
    args=ArgsPanel(method=('positive', 'normalized'))
)

`ArgsPanel` can be used the same way as `@interaction` decorator, except that it is specific to the `jsCommand`

### 1.3 Parameter labels
In out example, parameter names are not descriptive enough for general users. You can change the parameter name by passing a special argument `arglabels`, which is a `dict` parameter name:label as key:value 

In [ ]:
@interaction( method=('positive', 'normalized'), arglabels={'method':'FFT method'} )

### More customization
You can use input elements directly in the `args` of `jscommand`. Take look at the classes in [`JSinput` module](https://github.com/ahmohamed/nmrpro/blob/master/nmrpro/plugins/JSinput2.py).

## 2. Combining multiple functions in one `jsCommand` using `Include`
Creating a `jsCommand` for each python function will clutter the GUI and result in user confusion. We provide a way to `Include` mulitple function in a single command. A clear example for this the [`apodization`](https://github.com/ahmohamed/nmrpro/blob/master/nmrpro/plugins/apodization/apod.py#L114). 

In [ ]:
@jsCommand(['Processing', 'Apodization', 'Advanced apodization'], [1,2])
@interaction(inv=False, c=1., 
    em=(True, Include(EM)), gm=(False, Include(GM)), gmb=(False, Include(GMB)), 
    jmod=(False, Include(JMOD)), sp=(False, Include(SP)), tm=(False, Include(TM)),
    tri=(False, Include(TRI))
)

In this case, `EM`, `GM`, `GMB`, etc are window functions with changable parameters. Creating a `jsCommand` for each window function is clearly not a good idea. Instead, we want to create a single apodization command where users can select the window functions they want to apply and specify the corresponding paramters. 

The above `@interaction` creates panel with a checkbox for each function, when checked, it activates a set of inputs corresponding to the selected window function.

On the python side, if selected, `em`, `gm`, etc are set as callable functions. Simply, a call to `em`:

In [ ]:
em(spectrum)

will call the original function `EM` with the paramters selected by the user. Refer to [the code](https://github.com/ahmohamed/nmrpro/blob/master/nmrpro/plugins/apodization/apod.py#L114) for more details.

## 3. Specifying function order in the workflow
Processing functions for NMR spectra need to be in a certain order. For example, some functions such as zero filling, apodization need to be applied before FFT, while other need to be applied after. Baseline correction is applied as the last step. 

When users are writing processing script in python, it is the user responsibility to ensure that the order is correct. However, when users are interacting with a GUI, it is the developer's reponsibility to make sure that that each function is applied at the most sensible order. 

We provide a special decorator to specify where the function fit in the workflow. For example, in `apodization`:

In [ ]:
@forder(before=['FFT', 'ZF'])

which indicates that the function should be applied before FFT and zero filling.

The general syntax for `forder` is as follows:

In [ ]:
@forder(before=None, after=None, replaces=None, repeatable=False)

 - `before`, `after`: apply the function before of after certain processes.
 - `replaces`: list of conflicting functions. For example, covariance NMR is an alternative to FFT. Therefore, covariance should *replace* FFT.
 - `repeatable`: whether the function can be applied multiple times to the same spectrum. With few exceptions, such as baseline correction, functions are not repeatable. Set this to `True` if your function is repeatable.

## 4. R functions as plugins
To harness the power of R spectral process functionalities, NMRPro comes with module to facilitate this integration. We illustrate below with an example, using `baseline` function from the package `baseline`. Please note that you need R and the required packages installed beforehand. Also you need to install `rpy2` using `pip install`.

In [ ]:
from nmrpro.R import *
@R_function
def baseline(s, method='irls'):
    library('baseline')
    rmatrix = r.matrix(s, nrow=1) #baseline function requires input spectra as matrix
    
    bl = r.baseline(rmatrix, method=method)
    return r.getCorrected(bl)

First, we import `R` module from NMRPro, which contains four elements
 - `R_function` decorator: following the theme in plugin development, we use decorators to make plugin development easier. The decorator converts R vectors to NMRSpectrum objects seamlessly.
 - `library` function: imports packages in R.
 - `r` object: This is a python representation for R. All R variables and imported functions can be used by following the pattern `r.functionname`. In our example, `r.matrix`, `r.baseline` and `r.getCorrected` are R functions used directly in python.
 - `reval` function: Simply pass an R script as argument to get it *eval*uated in R. 

If you need additional advanced functionalities, you can refer to [rpy2 package documentation](http://rpy.sourceforge.net/rpy2/doc-dev/html/introduction.html)